# Building Multi-Agent Investment Firms with Strands SDK

## Introduction
In this challenge, participants will use the Strands Agents SDK to build a sophisticated multi-agent system that mirrors the dynamics of real-world trading firms. From fundamental analysts and sentiment experts to technical analysts, traders, and risk management teams, the system collaboratively evaluates market conditions and informs trading decisions.



## Task 1
- **Goal**: Be familiar with the basic use of the strands agent API and generate market analysis reports 
- Enter the date: 2025-08-19 and stock code:AMZN you wish to analyze
- Create two agents, market report agent and news report agent.
- Crawl stock historical data using market report agent.
- Crawl stock historical data using stock news agent.
- The final output will a csv files and text file.

## 1. Install dependecies
- If you encounter "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. ", you can ignore it

In [ ]:
%pip install -r requirements.txt

## 2. Prerequisites code  
### 2.1 Import Utils functions

In [3]:
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG
import logging
from dotenv import load_dotenv

### 2.2 Initiailize LLM models

In [4]:
NOVA_RPO_MODEL_ID = "amazon.nova-pro-v1:0"
EMBEDDING_MODEL_ID="amazon.titan-embed-text-v2:0"

### 2.3 Initiailize LLM models

In [5]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration with timeouts and retries
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes read timeout
    connect_timeout=900,    # 15 minutes connect timeout
    retries=dict(max_attempts=3, mode="adaptive"),
)


def get_model(model_id=NOVA_RPO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance.

    Args:
        model_id (str): The specific model ID to use
        thinking (bool): Whether to enable thinking mode for supported models
        temperature (float): Sampling temperature for response generation
        max_tokens (int): Maximum tokens in the response

    Returns:
        BedrockModel instance
    """
    session = boto3.Session()

    # Configure thinking mode for supported models
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

In [6]:
llm=get_model(NOVA_RPO_MODEL_ID)

### 2.4 Create market agent

The following function creates a Market Analyst agent that combines a language model with financial data tools to perform technical analysis and market trend evaluation. Depending on the online flag, it either uses real-time market data and indicators (via Yahoo Finance and technical reports) or switches to offline/cached data sources.

In [7]:
from strands import Agent
from tools import (
    get_yfin_data_online,
    get_stockstats_indicators_report_online,
    get_yfin_data,
    get_stockstats_indicators_report,
)

def create_market_analyst(llm, online=False):
    """
    Create a Market Analyst agent for technical analysis and market trend evaluation.
    
    Args:
        llm: The language model to use for the agent
        online (bool): Whether to use real-time data (True) or cached data (False)
        
    Returns:
        Agent: Configured Market Analyst agent
    """
    # Select tools based on online/offline mode
    if online:
        tools = [
            get_yfin_data_online,                    # Real-time Yahoo Finance data
            get_stockstats_indicators_report_online, # Real-time technical indicators
        ]
    else:
        tools = [
            get_yfin_data,                    # Cached Yahoo Finance data
            get_stockstats_indicators_report, # Cached technical indicators
        ]

    # Define the agent's system prompt with detailed instructions
    system_message = (
        """You are a Market Analyst specializing in technical analysis and market trend evaluation. 
        Your role is to analyze financial markets by selecting the **most relevant technical indicators** 
        for a given market condition or trading strategy.

        **Key Responsibilities:**
        1. Select up to **8 complementary indicators** that provide diverse insights without redundancy
        2. Analyze market trends, momentum, volatility, and volume patterns
        3. Provide actionable insights for trading decisions
        4. Identify support/resistance levels and trend directions

        **Available Technical Indicators by Category:**

        **Moving Averages:**
        - close_50_sma: 50-period Simple Moving Average for medium-term trend analysis
        - close_200_sma: 200-period SMA for long-term trend confirmation and golden/death cross setups
        - close_10_ema: 10-period Exponential Moving Average for short-term momentum shifts

        **MACD (Moving Average Convergence Divergence):**
        - macd: Main MACD line showing momentum via EMA differences
        - macds: MACD Signal line (smoothed EMA) for crossover signals
        - macdh: MACD Histogram showing momentum strength and early divergence signals

        **Momentum Indicators:**
        - rsi: Relative Strength Index for overbought/oversold conditions (70/30 thresholds)

        **Volatility Indicators:**
        - boll: Bollinger Bands Middle (20-period SMA baseline)
        - boll_ub: Bollinger Upper Band (overbought zones and breakout signals)
        - boll_lb: Bollinger Lower Band (oversold conditions)
        - atr: Average True Range for volatility measurement and risk management

        **Volume-Based Indicators:**
        - vwma: Volume Weighted Moving Average for trend confirmation with volume analysis

        **Analysis Guidelines:**
        1. Always call get_yfin_data first to retrieve the necessary price data
        2. Select indicators that provide complementary information (avoid redundancy)
        3. Explain why each selected indicator is suitable for the current market context
        4. Provide detailed, nuanced analysis rather than generic "mixed trends" statements
        5. Focus on actionable insights that help traders make informed decisions
        6. Include a markdown table at the end summarizing key findings

        **Output Format:**
        - Detailed technical analysis with specific observations
        - Clear trend identification and momentum assessment
        - Support/resistance level identification
        - Risk assessment and volatility analysis
        - Summary table with key metrics and recommendations
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        tools=tools,
        name="Market Analyst",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )

    return agent

### 2.5 Online analysis

Here we will use online analysis.

In [8]:
online=True
market_agent=create_market_analyst(llm, online)

### 2.6 Initiailize user input 

Set the target stock ticker `AMZN` and the specific trade date `2025-08-19` for analysis.

In [9]:
# Configuration
company_of_interest = "AMZN"  # Stock ticker to analyze
trade_date = "2025-08-19"    # Analysis date

Prepare `working_dir`, default location is the directory `results`.

In [10]:
working_dir = DEFAULT_CONFIG['results_dir']
prefix = f"{company_of_interest}_{trade_date}".replace(" ", "_")
config = DEFAULT_CONFIG.copy()

### 2.7 Starting analysis

In [11]:
print(f"Starting analysis for {company_of_interest} on {trade_date}")
prompt = f"Analyze the market for {company_of_interest} for the trade date {trade_date}"
result = market_agent(prompt)

Starting analysis for AMZN on 2025-08-19
<thinking> 
To analyze the market for AMZN, I will first retrieve the stock price data using the `get_yfin_data_online` tool. After obtaining the data, I will select up to 8 complementary indicators to provide diverse insights without redundancy. These indicators will cover moving averages, MACD, momentum, volatility, and volume-based analysis. 

Here are the indicators I plan to use:
1. close_50_sma - For medium-term trend analysis
2. close_200_sma - For long-term trend confirmation
3. close_10_ema - For short-term momentum shifts
4. macd - Main MACD line showing momentum via EMA differences
5. macdh - MACD Histogram showing momentum strength and early divergence signals
6. rsi - Relative Strength Index for overbought/oversold conditions
7. boll_ub and boll_lb - Bollinger Bands for volatility and breakout signals
8. atr - Average True Range for volatility measurement and risk management

I will then use the `get_stockstats_indicators_report_onl

### 2.8 Save analysis result 

In [12]:
def save_as_file(text, working_dir, prefix='', file_name=''):
    """
    Save text content to a file in the specified directory structure.
    
    Args:
        text (str): Content to save
        working_dir (str): Base working directory
        prefix (str): Subdirectory prefix (usually ticker_date)
        file_name (str): Name of the file to save
    """
    # Create directory if it doesn't exist
    full_dir = os.path.join(working_dir, prefix)
    if not os.path.exists(full_dir):
        os.makedirs(full_dir, exist_ok=True)
    
    # Write content to file
    file_path = os.path.join(full_dir, file_name)
    with open(file_path, "w", encoding='utf-8') as f:
        f.write(text)

In [13]:
save_as_file(str(result), working_dir, prefix, "market_report.txt")

## 3 Complete Task 1

Retrieve the stock data just generated, the directory is located `./results/data_cache/`.

In [14]:
stock_data_dir='./results/data_cache/'

In [18]:
fn=input("Enter the file name")

Enter the file name AMZN-YFin-data-2010-09-07-2025-09-07.csv


Display you just entered file name

In [19]:
fn

'AMZN-YFin-data-2010-09-07-2025-09-07.csv'

Display all data

In [20]:
import pandas as pd
df = pd.read_csv(stock_data_dir+fn)

In [21]:
df

,Date,Close,High,Low,Open,Volume
0,2010-09-07,6.861000,6.930000,6.844500,6.878000,77660000
1,2010-09-08,6.957000,6.985000,6.822500,6.896500,116570000
2,2010-09-09,7.019000,7.068500,6.963000,7.019500,99160000
3,2010-09-10,7.122000,7.130000,7.002000,7.037500,100872000
4,2010-09-13,7.253500,7.287000,7.188000,7.203500,101548000
...,...,...,...,...,...,...
3768,2025-08-29,229.000000,231.809998,228.160004,231.320007,26199200
3769,2025-09-02,225.339996,226.169998,221.830002,223.520004,38843900
3770,2025-09-03,225.990005,227.169998,224.360001,225.210007,29223100
3771,2025-09-04,235.679993,235.770004,230.779999,231.190002,59391800


Filter specified date stock information.

In [22]:
filtered_df = df[(df['Date'] == '2025-08-19') ]

In [23]:
filtered_df

,Date,Close,High,Low,Open,Volume
3760,2025-08-19,228.009995,230.529999,227.119995,230.089996,29891000


According above python script and complete `Task 1`

### 4 Continue to complete news analysis

The following function creates a News Analyst agent that uses a language model and selected news sources (real-time or cached) to analyze financial news, identify macroeconomic and geopolitical trends, assess market sentiment, and generate actionable insights for trading decisions based on a structured analysis framework.

In [25]:
from tools import (
    get_global_news_openai,
    get_google_news,
    get_finnhub_news,
    get_reddit_news,
)


def create_news_analyst(llm, online=False):
    """
    Create a News Analyst agent for news analysis and macroeconomic trend evaluation.
    
    Args:
        llm: The language model to use for the agent
        online (bool): Whether to use real-time news (True) or cached news (False)
        
    Returns:
        Agent: Configured News Analyst agent
    """
    # Select news sources based on online/offline mode
    if online:
        # Use real-time news sources for current analysis
        tools = [get_google_news]
    else:
        # Use cached news sources for historical analysis
        tools = [
            get_finnhub_news,    # Financial news from Finnhub
            get_reddit_news,     # Social sentiment from Reddit
            get_google_news,     # General news from Google
        ]

    # Define the agent's system prompt with comprehensive instructions
    system_message = (
        """You are a News Analyst specializing in financial news analysis and macroeconomic research. 
        Your role is to analyze recent news and global trends to provide context for trading decisions.

        **Key Responsibilities:**
        1. Analyze recent news and events from the past week that impact financial markets
        2. Identify macroeconomic trends and their potential market implications
        3. Assess geopolitical events and their effects on different sectors
        4. Evaluate market sentiment from news coverage and social media
        5. Provide actionable insights for trading strategies

        **Analysis Framework:**
        
        **Market-Moving Events:**
        - Central bank announcements and monetary policy changes
        - Economic data releases (GDP, inflation, employment, etc.)
        - Corporate earnings and guidance updates
        - Geopolitical developments and trade relations
        - Regulatory changes and policy announcements

        **Sector-Specific Impact:**
        - Technology: Innovation, regulation, competitive dynamics
        - Healthcare: Drug approvals, policy changes, demographic trends
        - Energy: Commodity prices, environmental policies, supply chain
        - Financial: Interest rates, regulatory changes, credit conditions
        - Consumer: Spending patterns, inflation impact, demographic shifts

        **Sentiment Analysis:**
        - Overall market sentiment (bullish, bearish, neutral)
        - Investor confidence indicators
        - Risk appetite and flight-to-quality movements
        - Social media sentiment and retail investor behavior

        **Global Context:**
        - International market developments
        - Currency movements and their implications
        - Cross-border trade and supply chain impacts
        - Regional economic conditions and policies

        **Analysis Guidelines:**
        1. Focus on news with clear market implications
        2. Distinguish between short-term noise and long-term trends
        3. Provide specific, actionable insights rather than generic observations
        4. Consider both direct and indirect effects on different asset classes
        5. Assess the reliability and credibility of news sources
        6. Include a markdown table summarizing key findings and implications

        **Output Requirements:**
        - Comprehensive analysis of market-relevant news and trends
        - Clear identification of potential trading opportunities and risks
        - Sector-specific impact assessment
        - Timeline of key events and their market implications
        - Summary table with key news items, impact assessment, and trading implications
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        tools=tools,
        name="News Analyst",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )

    return agent

In [26]:
news_analyst_agent = create_news_analyst(llm, online)

Run the news analysis for the target company on the given trade date, generate a report, save it to a file, and confirm completion.

In [28]:
print("Step 2: Running news analysis...")
prompt = f"Analyze the news for {company_of_interest} for the trade date {trade_date}"
result = news_analyst_agent(prompt)
save_as_file(str(result), working_dir,prefix, "news_report.txt")
print("News analysis completed.")

Step 2: Running news analysis...
<thinking> To analyze the news for Amazon (AMZN) for the trade date 2025-08-19, I will first gather the latest news related to Amazon from the past week. This will help me identify any market-moving events, sector-specific impacts, and overall market sentiment. </thinking> 
Tool #2: get_google_news
<thinking> Based on the retrieved news articles, I will now analyze the news for Amazon (AMZN) for the trade date 2025-08-19. I will identify market-moving events, sector-specific impacts, and overall market sentiment. </thinking>

### Amazon (AMZN) News Analysis for Trade Date 2025-08-19

#### Market-Moving Events:
1. **AWS and OpenAI Collaboration**:
   - **Event**: Amazon Web Services (AWS) announced that OpenAI open weight models are available via Amazon Bedrock and Amazon SageMaker AI.
   - **Implication**: This collaboration could enhance AWS's AI capabilities, potentially attracting more customers and increasing revenue. It may also positively impact A

### 5 Congraduations! Now you can continue to Challege 2.